In [2]:
!pip install pytorch_transformers --quiet
!pip install pytorch_pretrained_bert --quiet

     |████████████████████████████████| 184kB 2.7MB/s 
     |████████████████████████████████| 1.0MB 51.8MB/s 
     |████████████████████████████████| 890kB 50.9MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 


In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
 cd drive/My\ Drive/data

/content/drive/My Drive/data


In [33]:
!ls

bert_base_uncased  __init__.py	       __pycache__  train1.0.json
describe_data.py   my_process_json.py  README.md    util.py
dev1.0.json	   process_json.py     text.py


In [0]:
with open('./dev1.0.json') as f:
  dev_data = json.load(f)

In [35]:
len(dev_data['data'])

584

#### Print Example 1 in dev data

In [84]:
print (json.dumps(dev_data['data'][0], indent=2, sort_keys=True))

{
  "document": {
    "context": "Summary\n\nA male newborn was apparently well until his second day of life , when BEG__increased irritability__END and a BEG__swelling in his right leg__END were noted .\nHe was rooming - in with his mother since birth .\nOn BEG__examination__END , a BEG__mass__END on the anterior surface of the right leg was noticed .\nThe BEG__mass__END was firm , elongated , ill - defined , unmovable and BEG__painful__END at BEG__palpation__END .\nNo BEG__overlying skin changes__END were seen .\nThe newborn had a family history of BEG__neonatal bone swelling__END with resolution before the age of 2 .\nBEG__Subsequent images__END showed BEG__hyperostosis in the diaphysis of the right tibia__END .\nAfter exclusion of other conditions such as BEG__trauma__END , BEG__osteomyelitis__END and BEG__congenital syphilis__END , the BEG__involvement of the tibial diaphysis__END , sparing the epiphyses and the BEG__benign course of the disease__END in family history , were indic

In [0]:
import os
import json
import pandas as pd
import numpy as np
from pytorch_transformers import BertModel,BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import tqdm, trange


In [0]:
from my_process_json import *

In [0]:
 data_reader = MyDataReader('./'+ 'dev1.0.json', bs=50)
 mydata = data_reader.send_batches()

In [38]:
len(mydata)

50

In [0]:
#mydata[0]

In [40]:
dataset_size = data_reader.get_data_size()
dataset_size

50

In [0]:
paragraphs = [e['p'] for e in mydata].copy()
paragraph_tags = [e['p_tags'] for e in mydata].copy()
queries = [e['q'] for e in mydata].copy()
query_tags = [e['q_tags'] for e in mydata].copy()
answers = [e['a'] for e in mydata].copy()

#### The first 8 queries in Paragraphs are from Example 1. But the corresponding answers, queries and tags match correctly
##### Paragraph[0], answers[0], paragraph_tags[0] are from Example1.

In [42]:
paragraphs[0]

"caffey disease in neonatal period : the importance of the family ! summary  a male newborn was apparently well until his second day of life , when increased irritability and a swelling in his right leg were noted . he was rooming - in with his mother since birth . on examination , a mass on the anterior surface of the right leg was noticed . the mass was firm , elongated , ill - defined , unmovable and painful at palpation . no overlying skin changes were seen . the newborn had a family history of neonatal bone swelling with resolution before the age of 2 . subsequent images showed hyperostosis in the diaphysis of the right tibia . after exclusion of other conditions such as trauma , osteomyelitis and congenital syphilis , the involvement of the tibial diaphysis , sparing the epiphyses and the benign course of the disease in family history , were indicative of caffey disease . the genetic study confirmed this diagnosis . caffey disease , although rare , should not be overlooked in the

In [61]:
answers[0]

'caffey disease'

In [30]:
print(paragraph_tags[0])

['B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O

##### 2nd Paragraph is also from Example 1. 

In [72]:
paragraphs[1]

"caffey disease in neonatal period : the importance of the family ! summary  a male newborn was apparently well until his second day of life , when increased irritability and a swelling in his right leg were noted . he was rooming - in with his mother since birth . on examination , a mass on the anterior surface of the right leg was noticed . the mass was firm , elongated , ill - defined , unmovable and painful at palpation . no overlying skin changes were seen . the newborn had a family history of neonatal bone swelling with resolution before the age of 2 . subsequent images showed hyperostosis in the diaphysis of the right tibia . after exclusion of other conditions such as trauma , osteomyelitis and congenital syphilis , the involvement of the tibial diaphysis , sparing the epiphyses and the benign course of the disease in family history , were indicative of caffey disease . the genetic study confirmed this diagnosis . caffey disease , although rare , should not be overlooked in the

In [74]:
paragraph1 = paragraphs[1].split()
print(paragraph1)

['caffey', 'disease', 'in', 'neonatal', 'period', ':', 'the', 'importance', 'of', 'the', 'family', '!', 'summary', 'a', 'male', 'newborn', 'was', 'apparently', 'well', 'until', 'his', 'second', 'day', 'of', 'life', ',', 'when', 'increased', 'irritability', 'and', 'a', 'swelling', 'in', 'his', 'right', 'leg', 'were', 'noted', '.', 'he', 'was', 'rooming', '-', 'in', 'with', 'his', 'mother', 'since', 'birth', '.', 'on', 'examination', ',', 'a', 'mass', 'on', 'the', 'anterior', 'surface', 'of', 'the', 'right', 'leg', 'was', 'noticed', '.', 'the', 'mass', 'was', 'firm', ',', 'elongated', ',', 'ill', '-', 'defined', ',', 'unmovable', 'and', 'painful', 'at', 'palpation', '.', 'no', 'overlying', 'skin', 'changes', 'were', 'seen', '.', 'the', 'newborn', 'had', 'a', 'family', 'history', 'of', 'neonatal', 'bone', 'swelling', 'with', 'resolution', 'before', 'the', 'age', 'of', '2', '.', 'subsequent', 'images', 'showed', 'hyperostosis', 'in', 'the', 'diaphysis', 'of', 'the', 'right', 'tibia', '.', 

In [65]:
answers[1]

'infantile cortical hyperostosis'

In [68]:
print(paragraph_tags[1])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [80]:
#### To verify if the tags are correct for this paragraph
#### Find index of first occurence of answer in paragraph1 and verify that index in tags
index = [idx for idx, s in enumerate(paragraph1) if 'infanti' in s]
index

[1448, 1669, 1804, 2077, 2326, 2341, 2354]

In [81]:
#### First index is 1448
paragraph_tags[1][1448]

'B-ans'

#### 8th paragraph is also from Example 1

In [50]:
paragraphs[7]

"caffey disease in neonatal period : the importance of the family ! summary  a male newborn was apparently well until his second day of life , when increased irritability and a swelling in his right leg were noted . he was rooming - in with his mother since birth . on examination , a mass on the anterior surface of the right leg was noticed . the mass was firm , elongated , ill - defined , unmovable and painful at palpation . no overlying skin changes were seen . the newborn had a family history of neonatal bone swelling with resolution before the age of 2 . subsequent images showed hyperostosis in the diaphysis of the right tibia . after exclusion of other conditions such as trauma , osteomyelitis and congenital syphilis , the involvement of the tibial diaphysis , sparing the epiphyses and the benign course of the disease in family history , were indicative of caffey disease . the genetic study confirmed this diagnosis . caffey disease , although rare , should not be overlooked in the

In [64]:
queries[7]

'@placeholder , although rare , should not be forgotten in the diagnostic approach to early childhood bone swellings .'

In [62]:
answers[7]

'caffey disease'

In [63]:
print(paragraph_tags[7])

['B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ans', 'I-ans', 'O', 'O', 'O', 'O

In [59]:
queries[8]

're- @placeholder is a rare but potentially fatal condition which can occur while managing patients with pneumothorax or pleural effusion .'

##### 9th paragraph is from Example 2

In [82]:
paragraphs[8]

'fatal re- expansion pulmonary edema in a young adult following tube thoracostomy for spontaneous pneumothorax summary  re- expansion pulmonary edema ( rpe ) is a rare but potentially fatal complication that can occur following rapid lung expansion while managing patients with pleural effusion or pneumothorax . in this case , fatal outcome occurred due to rpe in a previously healthy young adult male patient subsequent to tube thoracostomy for spontaneous pneumothorax . while managing patients with pneumothorax or large pleural effusions , precautions should be taken to avoid rapid re-expansion of the previously collapsed lung in order to reduce the probability of development of this complication .  background  re- expansion pulmonary edema ( rpe ) is a rare complication which is attributed to the rapid drainage of air or fluid from the pleural cavity . it has been reported to occur following either thoracentesis or tube thoracostomy . the attributed pathological mechanism is the abrupt

##### So for every example of dev set, there are multiple paragraphs, queries and answers.